In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import yfinance as yf
import tensorflow as tf

ModuleNotFoundError: No module named 'requests'

In [2]:
pip install yfinance

  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
class StockAnalyzer:
    def __init__(self, ticker, start_date, end_date):
        self.df = yf.download(ticker, start=start_date, end=end_date)
        self.df.dropna(inplace=True)

    def visualize_stock_data(self):
        fig, (ax1, ax2, ax3) = plt.subplots(3, figsize=(24, 20))
        self.df.Close.plot(ax=ax1)
        ax1.set(title = f'{ticker} time series', ylabel = "Stock price")
        self.df.Volume.plot(ax=ax2)
        ax2.set(ylabel = "Volume")
        
        xval = np.arange(len(self.df))
        ax3.scatter(xval, self.df.Close.values)
        plt.show()

    def predict_price(self):
        X = self.df[['SMA', 'Upper', 'Lower']].dropna()
        y = self.df['Signal'][X.index]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        model = LinearRegression()
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)

        print("Mean Squared Error:", mean_squared_error(y_test, predictions))

    def visualize_moving_average_and_bollinger_bands(self, N=10, K=1.5):
        self.df['SMA'] = self.df['Close'].rolling(window=N).mean()
        self.df['STD'] = self.df['Close'].rolling(window=N).std()
        self.df['Upper'] = self.df['SMA'] + (self.df['STD'] * K)
        self.df['Lower'] = self.df['SMA'] - (self.df['STD'] * K)
        
        self.df['Signal'] = 0
        self.df.loc[self.df['Close'] > self.df['Upper'], 'Signal'] = -1  # Sell
        self.df.loc[self.df['Close'] < self.df['Lower'], 'Signal'] = 1   # Buy

        plt.figure(figsize=(12,6))
        self.df[['Close', 'SMA', 'Upper', 'Lower']].plot(ax=plt.gca())
        plt.title(f"{ticker} Moving Average and Bollinger Bands")
        plt.show()

if __name__ == "__main__":
    ticker = "AAPL"
    analyzer = StockAnalyzer(ticker, "2020-01-01", "2022-01-01")
    
    analyzer.visualize_stock_data()
    analyzer.visualize_moving_average_and_bollinger_bands()
    analyzer.predict_price()
